In [1]:
%%javascript
IPython.load_extensions('calico-document-tools');

<IPython.core.display.Javascript object>

# Imports

In [2]:
# Main imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
%matplotlib inline
from __future__ import division
from __future__ import print_function

# Needed for LineCollections helper functions
from matplotlib.collections import LineCollection

# Angle unit conversion
degtorad = np.pi/180.

# Define functions and classes to calculate and plot rays

In [12]:
#--------------------------------------------------------------------
# Main computational class for general rays
#--------------------------------------------------------------------

class rayThroughPlanarInterfaces():
    
    def __init__(self, ray_dist_from_axis=0.0, angle_rad=0.0, refr_index=1.0, start_location_on_axis=0.0):
        self.h = np.array([ray_dist_from_axis])
        self.ang_rad = np.array([angle_rad])
        self.refr_index = np.array([refr_index])
        self.location_on_axis = np.array([start_location_on_axis])
        self.normpower_in_rayTE = np.array([1.0])
        self.normpower_in_rayTM = np.array([1.0])
        self.normpower_in_rayUnpol = np.array([1.0])
        
    def propagate_to_and_thru_new_surface(self,l,new_refr_index):
        self.normpower_in_rayTE = np.append(self.normpower_in_rayTE, 
                                          self.normpower_in_rayTE[-1] * 
                                          transmittance_TE(self.ang_rad[-1],self.refr_index[-1],new_refr_index))
        self.normpower_in_rayTM = np.append(self.normpower_in_rayTM, 
                                          self.normpower_in_rayTM[-1] * 
                                          transmittance_TM(self.ang_rad[-1],self.refr_index[-1],new_refr_index))
        self.normpower_in_rayUnpol = 0.5 * (self.normpower_in_rayTE[-1] + self.normpower_in_rayTM[-1])
        self.location_on_axis = np.append(self.location_on_axis, self.location_on_axis[-1] + l)
        delta_h = l*np.tan(self.ang_rad[-1])
        self.h = np.append(self.h, self.h[-1] + delta_h)
        new_angle = np.arcsin(np.sin(self.ang_rad[-1])*self.refr_index[-1]/new_refr_index)
        self.ang_rad = np.append(self.ang_rad, new_angle)
        self.refr_index = np.append(self.refr_index, new_refr_index)
        
    def print_values(self):
        print('location_on_axis:', self.location_on_axis)
        print('               h:', self.h)
        print('         ang_rad:', self.ang_rad)
        print('      refr_index:', self.refr_index)
        print('normalized power')
        print('              TE:', self.normpower_in_rayTE)
        print('              TM:', self.normpower_in_rayTM)
        print('     Unpolarized:', self.normpower_in_rayUnpol)

#--------------------------------------------------------------------
# Utility functions to calculate reflectance/transmittance.
# Angle units are assumed to be in radians
#--------------------------------------------------------------------

def cos_angle2(angle1, n1, n2): # From Snell's law
    return np.sqrt(1 - (np.sin(angle1)*n1/n2)**2)

def reflectance_TE(angle1, n1, n2):
    c1 = np.cos(angle1)
    c2 = cos_angle2(angle1, n1, n2)
    temp = (n1*c1 - n2*c2) / (n1*c1 + n2*c2)
    return temp**2

def transmittance_TE(angle1, n1, n2):
    return 1.0 - reflectance_TE(angle1, n1, n2)

def reflectance_TM(angle1, n1, n2):
    c1 = np.cos(angle1)
    c2 = cos_angle2(angle1, n1, n2)
    temp = (n1*c2 - n2*c1) / (n1*c2 + n2*c1)
    return temp**2

def transmittance_TM(angle1, n1, n2):
    return 1.0 - reflectance_TM(angle1, n1, n2)

def reflectance_Unpol(angle1, n1, n2):
    rTE = reflectance_TE(angle1, n1, n2)
    rTM = reflectance_TM(angle1, n1, n2)
    return 0.5 * (rTE + rTM)

def transmittance_Unpol(angle1, n1, n2):
    return 1.0 - reflectance_Unpol(angle1, n1, n2)

#--------------------------------------------------------------------
# Helper functions to make a LineCollection
#   Topics: line, color, LineCollection, cmap, colorline, codex
#   From: http://nbviewer.ipython.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
#--------------------------------------------------------------------

'''
Defines a function colorline that draws a (multi-)colored 2D line with coordinates x and y.
The color is taken from optional data in z, and creates a LineCollection.

z can be:
- empty, in which case a default coloring will be used based on the position along the input arrays
- a single number, for a uniform color [this can also be accomplished with the usual plt.plot]
- an array of the length of at least the same length as x, to color according to this data
- an array of a smaller length, in which case the colors are repeated along the curve

The function colorline returns the LineCollection created, which can be modified afterwards.

See also: plt.streamplot
'''

# Data manipulation:

def make_segments(x, y):
    '''
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    '''

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    return segments


# Interface to LineCollection:

def colorline(ax, x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0), linewidth=3, alpha=1.0):
    '''
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    '''
    
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))
           
    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
        
    z = np.asarray(z)
    
    segments = make_segments(x, y)
    lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha)
    
    #ax = plt.gca()
    ax.add_collection(lc)
    
    return lc
        
    
def clear_frame(ax=None): 
    # Taken from a post by Tony S Yu
    if ax is None: 
        ax = plt.gca() 
    ax.xaxis.set_visible(False) 
    ax.yaxis.set_visible(False) 
    for spine in ax.spines.itervalues(): 
        spine.set_visible(False) 

#Define parameters, calculate rays 

In [13]:
thicknesses = np.array([0.01, 7.7, 4.0, 0.001])
refractive_indices = np.array([1.33, 1.46, 3.6, 3.6])
initial_refr_index = 1.33
start_location_on_axis = 0.0
array_of_rays = []
for i, angle in enumerate(range(-85,90,5)):
    array_of_rays.append(rayThroughPlanarInterfaces(ray_dist_from_axis = 0.0, 
                                                    angle_rad = angle*degtorad, 
                                                    refr_index = refractive_indices[0], 
                                                    start_location_on_axis = start_location_on_axis))
    for l,n in zip(thicknesses, refractive_indices):
        array_of_rays[i].propagate_to_and_thru_new_surface(l, n)

In [14]:
array_of_rays[0].print_values()

location_on_axis: [  0.00000000e+00   1.00000000e-02   7.71000000e+00   1.17100000e+01
   1.17110000e+01]
               h: [  0.          -0.11430052 -88.12570325 -96.76708031 -96.76747613]
         ang_rad: [-1.48352986 -1.48352986 -1.1372756  -0.37689867 -0.37689867]
      refr_index: [ 1.33  1.33  1.46  3.6   3.6 ]
normalized power
              TE: [ 1.          1.          0.53476985  0.27994383  0.27994383]
              TM: [ 1.          1.          0.60438031  0.6026235   0.6026235 ]
     Unpolarized: 0.441283664751
